In [1]:
#task input Creation:
from transformers import AutoTokenizer
import random
random.seed(13)
model_id = "Local-Meta-Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
def MakeRegressionTask(tokenizer,Context=False,max_examples_token_length=200):
    td=''

    weights=[random.randint(0, 100),random.randint(0, 100),random.randint(0, 100)]

    while True:
        inN=[random.randint(0, 100),random.randint(0, 100),random.randint(0, 100)]
        res=weights[0]*inN[0]+weights[1]*inN[1]+weights[2]*inN[2]
        td_new=td+'input = ( '+str(inN[0])+' , '+str(inN[1])+' , '+str(inN[2])+' ) ; output = '+str(res)+' \n'
        if tokenizer(td, return_tensors="pt").input_ids.shape[1]>max_examples_token_length:
            break
        else:
            td=td_new

    inN=[random.randint(0, 100),random.randint(0, 100),random.randint(0, 100)]
    res=weights[0]*inN[0]+weights[1]*inN[1]+weights[2]*inN[2]
    td=td+'input = ( '+str(inN[0])+' , '+str(inN[1])+' , '+str(inN[2])+' ) ; output = '
    if Context:
        td='The output represents the result of this linear equation given the input as the 3 input numbers: \n\n'+td
    return td,res
MakeRegressionTask(tokenizer,Context=True)

('The output represents the result of this linear equation given the input as the 3 input numbers: \n\ninput = ( 87 , 23 , 83 ) ; output = 10943 \ninput = ( 29 , 85 , 18 ) ; output = 5668 \ninput = ( 28 , 82 , 93 ) ; output = 12049 \ninput = ( 23 , 16 , 9 ) ; output = 2134 \ninput = ( 68 , 27 , 95 ) ; output = 11508 \ninput = ( 37 , 3 , 55 ) ; output = 6117 \ninput = ( 16 , 87 , 77 ) ; output = 10446 \ninput = ( 1 , 35 , 18 ) ; output = 2894 \ninput = ( 10 , 33 , 57 ) ; output = 6510 \ninput = ( 95 , 55 , 17 ) ; output = 6649 \ninput = ( 32 , 45 , 29 ) ; output = 5244 \ninput = ( 72 , 54 , 85 ) ; output = ',
 11769)

In [3]:
f=open('./Datasets/positive-words.txt',"r")
pos_words=f.read().split('\n')[:-1]
f.close()
f=open('./Datasets/negative-words.txt',"r",encoding="ISO-8859-1")
neg_words=f.read().split('\n')[:-1]
f.close()

def MakeClassificationTask(tokenizer,Context=False,max_examples_token_length=200):
    td=''
    
    while True:

        if random.randint(0, 1)==1:
            td_new=td+'input = '+pos_words[random.randint(0, len(pos_words)-1)]+' ; output = positiv \n'
        else:
            td_new=td+'input = '+neg_words[random.randint(0, len(neg_words)-1)]+' ; output = negativ \n'
        if tokenizer(td, return_tensors="pt").input_ids.shape[1]>max_examples_token_length:
            break
        else:
            td=td_new

    res=None
    if random.randint(0, 1)==1:
        td=td+'input = '+pos_words[random.randint(0, len(pos_words)-1)]+' ; output = '
        res='positiv'
    else:
        td=td+'input = '+neg_words[random.randint(0, len(neg_words)-1)]+' ; output = '
        res='negativ'
    if Context:
        td='The following words are classified by the sentiment they imply: \n\n'+td
    return td,res

MakeClassificationTask(tokenizer,Context=True)

('The following words are classified by the sentiment they imply: \n\ninput = stringently ; output = negativ \ninput = plush ; output = positiv \ninput = compliant ; output = positiv \ninput = restructure ; output = positiv \ninput = titillating ; output = positiv \ninput = delicacy ; output = positiv \ninput = laudable ; output = positiv \ninput = harboring ; output = negativ \ninput = dexterous ; output = positiv \ninput = wrest ; output = negativ \ninput = mocks ; output = negativ \ninput = courageousness ; output = positiv \ninput = dawn ; output = positiv \ninput = unsuccessful ; output = negativ \ninput = gimmick ; output = negativ \ninput = cheesy ; output = negativ \ninput = politeness ; output = positiv \ninput = rip-off ; output = negativ \ninput = positive ; output = positiv \ninput = traumatized ; output = negativ \ninput = detriment ; output = negativ \ninput = exceeds ; output = ',
 'positiv')

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import infer_auto_device_map, init_empty_weights
from torch.utils.checkpoint import checkpoint
from torch.cuda.amp import autocast, GradScaler

model_id = "Local-Meta-Llama-3.2-1B"
with init_empty_weights():
    my_model = AutoModelForCausalLM.from_pretrained(model_id)
device_map = infer_auto_device_map(my_model, max_memory={0: "1GiB", "cpu": "30GiB"})

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,device_map=device_map)
model.gradient_checkpointing_enable()


#def zero_out_grad_hook(grad):
#    return torch.zeros_like(grad) 
#for param in model.parameters():
#    param.register_hook(lambda grad: None)

def move_grad_to_cpu_hook(grad):
    return grad.cpu()  # Move the gradient to CPU (RAM)

# Register the hook for all parameters in the model
#for param in model.parameters():
#     param.requires_grad = False
for param in model.lm_head.parameters():  # Unfreeze the output layer
    param.requires_grad = True

def move_grad_to_cpu(grad):
    return grad.to('cpu')  # Move the gradient to CPU

# Register the hook on each parameter
for param in model.parameters():
    param.register_hook(move_grad_to_cpu)

/home/cl2/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:2068: UserWarning: for model.embed_tokens.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/cl2/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:2068: UserWarning: for model.layers.0.self_attn.q_proj.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(f'for {key}: copying from a non-meta parameter in the checkpoint to a meta '
/home/cl2/anaconda3/l

In [2]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [3]:
#model.model.layers[2].self_attn.k_proj
layers_to_hook = {}
layers_to_hook["embed_tokens"]=[model.model.embed_tokens]
for i in model.model.layers:
    
    if "q_proj" not in layers_to_hook:
        layers_to_hook["q_proj"]=[]
    layers_to_hook["q_proj"].append(i.self_attn.q_proj)
    
    if "k_proj" not in layers_to_hook:
        layers_to_hook["k_proj"]=[]
    layers_to_hook["k_proj"].append(i.self_attn.k_proj)
    
    if "v_proj" not in layers_to_hook:
        layers_to_hook["v_proj"]=[]
    layers_to_hook["v_proj"].append(i.self_attn.v_proj)
    
    if "o_proj" not in layers_to_hook:
        layers_to_hook["o_proj"]=[]
    layers_to_hook["o_proj"].append(i.self_attn.o_proj)
    
    if "rotary_emb" not in layers_to_hook:
        layers_to_hook["rotary_emb"]=[]
    layers_to_hook["rotary_emb"].append(i.self_attn.rotary_emb)
    
    if "mlp" not in layers_to_hook:
        layers_to_hook["mlp"]=[]
    layers_to_hook["mlp"].append(i.mlp)


layers_to_hook["embed_tokens"]=[model.model.rotary_emb]

In [4]:
# Dictionary to store the output of each hooked layer and its gradient
extracted_outputs = {}

def move_to_cpu_with_grad(data):
    if isinstance(data, torch.Tensor):  # Check if it's a tensor
        # Move to CPU, ensure requires_grad is True, and retain gradients
        data = data.to('cpu').requires_grad_(True)
        data.retain_grad()  # Retain gradients for non-leaf tensors
        return data
    elif isinstance(data, dict):  # If it's a dictionary, recursively check its values
        return {key: move_to_cpu_with_grad(value) for key, value in data.items()}
    elif isinstance(data, list):  # If it's a list, recursively check each element
        return [move_to_cpu_with_grad(item) for item in data]
    elif isinstance(data, tuple):  # If it's a tuple, recursively check each element
        return tuple(move_to_cpu_with_grad(item) for item in data)
    else:
        return data  # If it's not a tensor, return it as-is

# Hook function factory that returns a hook function for each layer
def create_hook_fn(layer_name,layer_index):
    def hook_fn(module, input, output):
        if layer_name not in extracted_outputs:
            extracted_outputs[layer_name] = {}
        extracted_outputs[layer_name][layer_index]=move_to_cpu_with_grad(output)
    return hook_fn

# Example: Hooking multiple layers (k_proj and v_proj in the first transformer layer)
# You can add more layers similarly by changing the names and layers.

# Register hooks for the desired layers
hooks = []
for layer_name, layer_arr in layers_to_hook.items():
    for layer_pos,layer in enumerate(layer_arr):
        hook = layer.register_forward_hook(create_hook_fn(layer_name,layer_pos))
        hooks.append(hook)

In [5]:
text = "This is a test sentence"
tokenized_output = tokenizer(text, return_tensors="pt")
tokenized_output = {key: value.to('cpu') for key, value in tokenized_output.items()}
with torch.autograd.graph.save_on_cpu():
    with autocast():
        M_outputs = model(**tokenized_output)  # Perform the forward pass without storing parameter gradients

/tmp/ipykernel_177892/2876764701.py:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [6]:
print(extracted_outputs.keys())
print(M_outputs.logits[0][-1][398])

dict_keys(['embed_tokens', 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'mlp'])
tensor(7.5657, grad_fn=<SelectBackward0>)


In [7]:
loss = M_outputs.logits[0][-1][23987] # Use the k_proj output for backward pass
model.zero_grad()
#loss.backward()
GradScaler().scale(loss).backward()

/tmp/ipykernel_177892/501562521.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  GradScaler().scale(loss).backward()


In [11]:
extracted_outputs["embed_tokens"][0][0].grad

tensor([[[ 2.2605e+01,  2.6987e+01,  2.9781e+01, -1.3537e+02,  4.2770e+01,
          -1.1903e+01, -1.3949e+01, -5.1854e+02, -7.2073e+02, -1.3281e+02,
           1.8215e+02, -1.6404e+03,  2.5903e+01, -9.7003e+02, -6.5150e+02,
          -5.8118e+02, -4.9534e+03, -9.1092e+03, -5.3643e+03, -1.8437e+03,
          -3.1328e+03, -8.3125e+03, -2.0272e+03, -1.6877e+04, -1.0086e+04,
          -2.9641e+03, -8.5136e+03,  7.0061e+03, -4.7000e+03, -5.9806e+03,
          -1.5357e+04, -5.1297e+04, -1.0618e+01, -4.7002e+01, -1.5457e+02,
          -6.9218e+01,  5.0785e+01, -9.5006e+01, -7.7247e+01,  1.9884e+02,
          -2.6148e+02,  1.2428e+02, -8.4040e+02, -2.4184e+03,  1.6465e+02,
          -1.5038e+03, -2.6871e+03, -9.2840e+02, -3.6808e+02, -6.8971e+02,
          -3.3897e+03, -7.2401e+03, -2.1939e+04, -6.7925e+03, -9.1170e+03,
          -5.8670e+03, -5.1162e+03,  1.3867e+04,  4.4069e+03, -1.5600e+03,
          -1.3662e+03, -1.5843e+04, -4.6871e+03,  8.2495e+03],
         [-4.2099e+03,  3.7992e+03,  

In [ ]:
#Experiments until here 

In [ ]:
# Access gradients for the hooked layers
k_proj_output_grad = extracted_outputs["k_proj_output"].grad
v_proj_output_grad = extracted_outputs["v_proj_output"].grad

# Remove the hooks (optional)
for hook in hooks:
    hook.remove()

# Print gradients for the hooked layers
print("Gradient of k_proj output:", k_proj_output_grad)
print("Gradient of v_proj output:", v_proj_output_grad)rotary_emb

In [ ]:


# Find the k_proj layer in the attention mechanism (first transformer layer here)
layer = model.encoder.layer[0].attention.self.key  # Key projection layer (k_proj)

# Register the forward hook
hook = layer.register_forward_hook(hook_fn)

# Tokenize the input
text = "This is a test sentence."
inputs = tokenizer(text, return_tensors="pt")

# Perform forward pass
outputs = model(**inputs)

# Now, the output of k_proj has been captured and retain_grad() called
k_proj_output = extracted_outputs["k_proj_output"]

# We can create a scalar loss or sum of the final output to perform backward on
# For simplicity, let's sum the outputs of the last hidden state (you can replace this with any loss function)
loss = outputs.last_hidden_state.sum()

# Perform the backward pass to compute gradients
loss.backward()

# Access the gradient of the k_proj output after backward pass
k_proj_output_grad = k_proj_output.grad

# Remove the hook (optional)
hook.remove()

# Print the gradient for k_proj output
print("Gradient of k_proj output:", k_proj_output_grad)


In [14]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

for param in model.parameters():
   param.requires_grad = False  # Disable gradient tracking

# Dictionary to store the output of k_proj and its gradient
extracted_outputs = {}



# Hook function to capture the output of the k_proj layer
def create_hook_fn(layer_name):
    def hook_fn(module, input, output):
        extracted_outputs[layer_name] = output.to(device="cpu")
        output.requires_grad_(True)
        output.retain_grad()  # Retain gradients for this layer's output only
    return hook_fn

# Example: Hooking multiple layers (k_proj and v_proj in the first transformer layer)
# You can add more layers similarly by changing the names and layers.
layers_to_hook = {
    "k_proj_output": model.encoder.layer[0].attention.self.key,  # Key projection layer
    "v_proj_output": model.encoder.layer[0].attention.self.value  # Value projection layer
}

# Tokenize the input
text = "This is a test sentence."
inputs = tokenizer(text, return_tensors="pt")


# Find the k_proj layer in the attention mechanism (first transformer layer here)
layer = model.encoder.layer[0].attention.self.key  # Key projection layer (k_proj)

hooks = []
for layer_name, layer in layers_to_hook.items():
    hook = layer.register_forward_hook(create_hook_fn(layer_name))
    hooks.append(hook)

outputs = model(**inputs)  # Perform the forward pass without storing parameter gradients



# Re-enable gradient computation for intermediate layers
k_proj_output = extracted_outputs["v_proj_output"]

# Enable gradient computation for k_proj output only
k_proj_output.requires_grad_(True)  # Ensure gradients are enabled for the k_proj output

# Create a scalar loss or sum of the final output to perform backward on
loss = k_proj_output.sum()  # Use k_proj output directly for backward pass

# Perform the backward pass to compute gradients
loss.backward()

# Access the gradient of the k_proj output after the backward pass
k_proj_output_grad = k_proj_output.grad

# Remove the hook (optional)
hook.remove()

# Print the gradient for k_proj output
print("Gradient of k_proj output:", k_proj_output_grad)
print(extracted_outputs)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Gradient of k_proj output: tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])
{'k_proj_output': tensor([[[ 1.2797e+00,  2.2044e-01,  2.4081e-01,  ..., -1.7203e-02,
          -2.6863e-01,  7.2731e-01],
         [ 3.5969e-01, -4.7975e-01, -9.1391e-02,  ..., -8.5932e-01,
           2.7313e-01, -2.7672e+00],
         [ 5.3447e-01, -5.9713e-01,  2.5200e-03,  ..., -1.1064e+00,
          -3.9094e-01, -3.2550e+00],
         ...,
         [-3.1752e-01, -6.9256e-01, -3.3283e-01,  ..., -1.9510e+00,
          -1.2154e-01, -2.9600e+00],
         [ 1.0200e-02,  1.6724e-01, -8.6441e-02,  ...,  2.2651e+00,
          -1.1164e+00, -3.7025e+00],
         [-6.6954e-01,  1.0858e+00,  6.5653e-02,  ..., -6.1973e-02,
          -1.1029e+00, -2.5208e+00]]], grad_fn=<ViewBackward0>), 'v_proj_output': tens

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Dictionary to store the output of each hooked layer and its gradient
extracted_outputs = {}

# Hook function factory that returns a hook function for each layer
def create_hook_fn(layer_name):
    def hook_fn(module, input, output):
        extracted_outputs[layer_name] = output
        output.retain_grad()  # Retain gradients for this layer's output only
    return hook_fn

# Example: Hooking multiple layers (k_proj and v_proj in the first transformer layer)
# You can add more layers similarly by changing the names and layers.
layers_to_hook = {
    "k_proj_output": model.encoder.layer[0].attention.self.key,  # Key projection layer
    "v_proj_output": model.encoder.layer[0].attention.self.value  # Value projection layer
}

# Register hooks for the desired layers
hooks = []
for layer_name, layer in layers_to_hook.items():
    hook = layer.register_forward_hook(create_hook_fn(layer_name))
    hooks.append(hook)

# Tokenize the input
text = "This is a test sentence."
inputs = tokenizer(text, return_tensors="pt")

# Perform the forward pass (with hooks capturing intermediate outputs)
with torch.no_grad():
    outputs = model(**inputs)  # Forward pass without storing gradients for parameters

# Re-enable gradient computation for the extracted outputs
for layer_name, output in extracted_outputs.items():
    output.requires_grad_(True)  # Enable gradient computation for the extracted outputs

# Example: Use the output of "k_proj_output" for backward pass
k_proj_output = extracted_outputs["k_proj_output"]
loss = k_proj_output.sum()  # Use the k_proj output for backward pass
loss.backward()

# Access gradients for the hooked layers
k_proj_output_grad = extracted_outputs["k_proj_output"].grad
v_proj_output_grad = extracted_outputs["v_proj_output"].grad

# Remove the hooks (optional)
for hook in hooks:
    hook.remove()

# Print gradients for the hooked layers
print("Gradient of k_proj output:", k_proj_output_grad)
print("Gradient of v_proj output:", v_proj_output_grad)
